In [5]:
import sys, os
sys.path.append('C:/myStudy/ZeroKaraTsukuru/deep-learning-from-scratch-master')
from common.functions import *
from common.gradient import numerical_gradient

import numpy as np

|변수|설명|
|--|--|
|params|• 신경망의 매개변수를 보관하는 딕셔너리 변수(인스턴스 변수)<br>• params['W1']은 1번째 층의 가중치, params['b1']은 1번째 층의 편향|
|grads|• 기울기를 보관하는 딕셔너리 변수(numerical_gradient() 메서드의 반환 값)<br>• grads['W1']은 첫 번쨰 층의 가중치의 기울기, grads['b1']은 1번째 층의 편향의 기울기|

---
|메서드|설명|
|--|--|
|\__ init \__(self, input_size,<br>hidden_size, output_size)|초기화를 수행한다.<br>인수는 순서대로 입력층의 뉴런 수, 은닉층의 뉴런 수, 출력층의 뉴런 수|
|predict(x)|예측(추론)을 수행한다.<br>인수 x는 이미지 데이터|
|loss(self, x, t)|손실함수의 값을 구한다.<br>인수 x는 이미지 데이터, t는 정답 레이블|
|accuracy(self, x, t)|정확도를 구한다.|
|numerical_gradient(self, x, t)|가중치 매개변수의 기울기를 구한다.|
|gradient(self, x, t)|가중치 매개변수의 기울기를 구한다.<br>numerical_gradient()의 성능 개선판|


In [6]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        return y
    
    def loss(self, x, t):
        y = self.predict(x)

        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        accuracy = np.sum(y == t) / float(x.shape[0])

        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}

        batch_num = x.shape[0]

        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, b1)
        grads['b2'] = np.sum(dy, axis=0)

        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads

In [7]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)

print(net.params['W1'].shape)
print(net.params['b1'].shape)
print(net.params['W2'].shape)
print(net.params['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


In [8]:
x = np.random.rand(100, 784)
y = net.predict(x)

print(y)

[[0.09496862 0.09419766 0.09499724 0.10239416 0.10421688 0.10285519
  0.09932903 0.10442651 0.09979416 0.10282055]
 [0.09500509 0.09392375 0.09490394 0.10244174 0.10422773 0.10279949
  0.0993802  0.10463703 0.09957933 0.1031017 ]
 [0.0952609  0.09412494 0.09483403 0.10240385 0.10402732 0.10268943
  0.09961748 0.10483685 0.09961249 0.10259272]
 [0.09524618 0.09395565 0.09504555 0.10250047 0.10395416 0.10305639
  0.09947914 0.10439835 0.09991598 0.10244813]
 [0.09533587 0.09397313 0.09480702 0.10232798 0.10403018 0.10292264
  0.09914702 0.10472218 0.10008913 0.10264486]
 [0.0954019  0.09388852 0.09483715 0.10239893 0.10405286 0.10293437
  0.09948846 0.10471109 0.09973691 0.1025498 ]
 [0.09532988 0.09374457 0.09455597 0.10242748 0.10386316 0.10297687
  0.09962231 0.10471644 0.10001989 0.10274343]
 [0.0952367  0.09446798 0.09485073 0.10235793 0.10395538 0.10297279
  0.09946762 0.10460542 0.09969444 0.10239101]
 [0.09499538 0.09411498 0.09460816 0.10273639 0.10410342 0.10296956
  0.09937835